In [39]:
import requests as re
from ipyleaflet import Map, basemaps, basemap_to_tiles, TileLayer

STAC_ENDPOINT_URL = "https://staging-stac.delta-backend.xyz"
RASTER_ENDPOINT_URL = "https://staging-raster.delta-backend.xyz"

In [71]:
no2_collection_summary = re.get(f"{STAC_ENDPOINT_URL}/collections/no2-monthly").json()
no2_collection_summary

{'id': 'no2-monthly',
 'type': 'Collection',
 'links': [{'rel': 'items',
   'type': 'application/geo+json',
   'href': 'https://staging-stac.delta-backend.xyz/collections/no2-monthly/items'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://staging-stac.delta-backend.xyz/'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://staging-stac.delta-backend.xyz/'},
  {'rel': 'self',
   'type': 'application/json',
   'href': 'https://staging-stac.delta-backend.xyz/collections/no2-monthly'}],
 'title': 'NO₂',
 'extent': {'spatial': {'bbox': [[-180, -90, 180, 90]]},
  'temporal': {'interval': [['2016-01-01T00:00:00Z',
     '2022-01-01T00:00:00Z']]}},
 'license': 'MIT',
 'summaries': {'datetime': ['2016-01-01T00:00:00Z', '2022-05-01T00:00:00Z'],
  'cog_default': {'max': 50064805976866816, 'min': -6618294421291008}},
 'description': 'Darker colors indicate higher nitrogen dioxide (NO₂) levels and more activity. Lighter colors indicate lower levels of NO₂ an

In [72]:
rescale_values = no2_collection_summary["summaries"]["cog_default"]
rescale_values

{'max': 50064805976866816, 'min': -6618294421291008}

In [61]:
response = re.post(
    f"{STAC_ENDPOINT_URL}/search", 
    json={
        "collections": ["no2-monthly"], 
        "query": {"datetime": {"eq":"2021-01-01T00:00:00"}},
        "limit": 100
    }
).json()
item = response["features"][0]
item

In [77]:
tiles = re.get(
    f"{RASTER_ENDPOINT_URL}/stac/tilejson.json?collection={item['collection']}&item={item['id']}&assets=cog_default&color_formula=gamma+r+1.05&colormap_name=rdbu_r&rescale=0%2C15000000000000000", 
).json()
tiles

{'tilejson': '2.2.0',
 'version': '1.0.0',
 'scheme': 'xyz',
 'tiles': ['https://staging-raster.delta-backend.xyz/stac/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?collection=no2-monthly&item=OMI_trno2_monthly_0.10x0.10_202101_Col3_V4.nc&assets=cog_default&color_formula=gamma+r+1.05&colormap_name=rdbu_r&rescale=0%2C15000000000000000'],
 'minzoom': 0,
 'maxzoom': 24,
 'bounds': [-180.0, -90.0, 180.0, 90.0],
 'center': [0.0, 0.0, 0]}

In [78]:
m = Map(
    zoom=2,
    scroll_wheel_zoom=True
)
no2_layer = TileLayer(url=tiles["tiles"][0], minzoom=18, maxzoom=0, show_loading=True)

m.add_layer(no2_layer)
m

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…